In [1]:
from scraping import Reg_API, CommentParser

In [2]:
api = Reg_API()

In [3]:
api.endpoint("/comments")

In [4]:
api.search("climate")

In [5]:
res = api.get()

In [6]:
api.endpoint("/documents/FS-2023-0006-0002")

In [7]:
api.get()

{'data': {'id': 'FS-2023-0006-0002',
  'type': 'documents',
  'links': {'self': 'https://api.regulations.gov/v4/documents/FS-2023-0006-0002'},
  'attributes': {'additionalRins': None,
   'allowLateComments': False,
   'authorDate': None,
   'authors': None,
   'cfrPart': '36 CFR Part 200',
   'commentEndDate': '2023-07-21T03:59:59Z',
   'commentStartDate': '2023-04-21T04:00:00Z',
   'effectiveDate': None,
   'exhibitLocation': None,
   'exhibitType': None,
   'frDocNum': '2023-08429',
   'frVolNum': None,
   'implementationDate': None,
   'media': None,
   'ombApproval': None,
   'paperLength': 0,
   'paperWidth': 0,
   'regWriterInstruction': None,
   'sourceCitation': None,
   'startEndPage': '24497 - 24503',
   'subject': None,
   'topics': None,
   'address1': None,
   'address2': None,
   'agencyId': 'FS',
   'city': None,
   'category': None,
   'comment': None,
   'country': None,
   'displayProperties': None,
   'docAbstract': None,
   'docketId': 'FS-2023-0006',
   'documentTy

In [8]:
res["data"][0]

{'id': 'PHMSA-2019-0100-2389',
 'type': 'comments',
 'attributes': {'documentType': 'Public Submission',
  'lastModifiedDate': '2019-08-06T13:57:16Z',
  'highlightedContent': 'Comment from <mark><em>Climate</em></mark> Writers',
  'withdrawn': False,
  'agencyId': 'PHMSA',
  'title': 'Comment from Climate Writers',
  'objectId': '0900006483dd9a06',
  'postedDate': '2019-08-02T04:00:00Z'},
 'links': {'self': 'https://api.regulations.gov/v4/comments/PHMSA-2019-0100-2389'}}

In [33]:
api.endpoint("/comments/PHMSA-2019-0100-2389")

In [34]:
res = api.get()

In [11]:
type(res)

dict

In [12]:
res.keys()

dict_keys(['data'])

In [22]:
res["data"].keys()

dict_keys(['id', 'type', 'links', 'attributes', 'relationships'])

In [36]:
res["data"]["attributes"]["fileFormats"]

In [41]:
for k in res["data"]["attributes"]:
    print(k,res["data"]["attributes"][k])

commentOn 0900006483cf03d6
commentOnDocumentId PHMSA-2019-0100-0002
duplicateComments 0
address1 None
address2 None
agencyId PHMSA
city None
category None
comment attach
country None
displayProperties None
docAbstract None
docketId PHMSA-2019-0100
documentType Public Submission
email None
fax None
field1 None
field2 None
fileFormats None
firstName Climate
govAgency None
govAgencyType None
objectId 0900006483dd9a06
lastName Writers
legacyId None
modifyDate 2019-08-06T13:57:16Z
organization None
originalDocumentId None
pageCount 0
phone None
postedDate 2019-08-02T04:00:00Z
postmarkDate None
reasonWithdrawn None
receiveDate 2019-07-31T04:00:00Z
restrictReason None
restrictReasonType None
stateProvinceRegion None
submitterRep None
submitterRepAddress None
submitterRepCityState None
subtype Comment(s)
title Comment from Climate Writers
trackingNbr 1k3-9bcj-wytd
withdrawn False
zip None
openForComment False


In [48]:
res["data"]["relationships"]

{'attachments': {'links': {'self': 'https://api.regulations.gov/v4/comments/PHMSA-2019-0100-2389/relationships/attachments',
   'related': 'https://api.regulations.gov/v4/comments/PHMSA-2019-0100-2389/attachments'}}}

In [28]:
id = "FS-2023-0006-65833"
api.endpoint(f"/comments/{id}")

In [29]:
res = api.get()

In [32]:
res["data"]["attributes"]["fileFormats"]

In [52]:
id = "FS-2023-0006-65833"
api.endpoint(f"/comments/{id}")
res = api.get()

In [53]:
api.url(res["data"]["relationships"]["attachments"]["links"]['related']).get()["data"]

[]

In [ ]:
comments = []
N = 999
p = 1
api.endpoint("/comments")
api.search("climate")
api.page(2)
res = api.get()
i = 1
for entry in res["data"]:
    print(entry)
    

    # has_attachments = api.url(res["data"]["relationships"]["attachments"]["links"]['related']).get()["data"]
    # i += 1
    # if not has_attachments:
    #     text = res["data"]["attributes"]["comment"]
    #     comments.append(text)


    

{'id': 'BOEM-2024-0022-0054', 'type': 'comments', 'attributes': {'documentType': 'Public Submission', 'lastModifiedDate': '2024-07-04T00:33:53Z', 'highlightedContent': 'Rogue <mark><em>Climate</em></mark> _ORPSN_ 06_28_2024&hellip;&nbsp;Rogue <mark><em>Climate</em></mark> serves and works alongside Southern Oregon and South Coast communities most\n\nimpacted by&hellip;&nbsp;<mark><em>climate</em></mark> change, including low-income, rural, youth, and communities of color, by\n\norganizing for clean&hellip;&nbsp;Rogue <mark><em>Climate</em></mark> is invested in the protection of the Coos Bay estuary and the <mark><em>climate</em></mark> resilience of\n\nSouth&hellip;&nbsp;Ensuring A Just Transition:\nThrough the <mark><em>Climate</em></mark> Justice Alliance, Rogue <mark><em>Climate</em></mark>, alongside UPROSE and&hellip;&nbsp;Rogue <mark><em>Climate</em></mark> looks forward to continued collaboration and\n\nengagement into the future.', 'withdrawn': False, 'agencyId': 'BOEM', 'titl

In [61]:
api.endpoint("/comments")
api.search("climate")
api.page(2)
res = api.get()
res["data"]


[{'id': 'BOEM-2024-0022-0054',
  'type': 'comments',
  'attributes': {'documentType': 'Public Submission',
   'lastModifiedDate': '2024-07-04T00:33:53Z',
   'highlightedContent': 'Rogue <mark><em>Climate</em></mark> _ORPSN_ 06_28_2024&hellip;&nbsp;Rogue <mark><em>Climate</em></mark> serves and works alongside Southern Oregon and South Coast communities most\n\nimpacted by&hellip;&nbsp;<mark><em>climate</em></mark> change, including low-income, rural, youth, and communities of color, by\n\norganizing for clean&hellip;&nbsp;Rogue <mark><em>Climate</em></mark> is invested in the protection of the Coos Bay estuary and the <mark><em>climate</em></mark> resilience of\n\nSouth&hellip;&nbsp;Ensuring A Just Transition:\nThrough the <mark><em>Climate</em></mark> Justice Alliance, Rogue <mark><em>Climate</em></mark>, alongside UPROSE and&hellip;&nbsp;Rogue <mark><em>Climate</em></mark> looks forward to continued collaboration and\n\nengagement into the future.',
   'withdrawn': False,
   'agencyI